## Branch and Relations Manager Dimension Tables

#### Mount `datatechstorage` Data Lake Storage

In [0]:
# the first piece of info needed is the container and storage account name.
storage_account = "datatechstorage"
container = "users"

# now form blobEnpoint
config_key = f"fs.azure.account.key.{storage_account}.blob.core.windows.net"
blobEndpoint = f"wasbs://{container}@{storage_account}.blob.core.windows.net"
mount_point = f"/mnt/{storage_account}/{container}"

dbutils.fs.mount(
    source = blobEndpoint
    ,mount_point = mount_point
    ,extra_configs = {config_key:dbutils.secrets.get(scope = "datatech", key = "access-adls") }
  )
print(mount_point)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2878635156727341> in <module> 
 8 mount_point = f"/mnt/{storage_account}/{container}" 
 9 
 ---> 10 dbutils.fs.mount(
 11 source = blobEndpoint
 12 , mount_point = mount_point

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling o273.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datatechstorage/users; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datatechstorage/users
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:739)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datatechstorage/users
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:418)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:714)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:518)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:703)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:426)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:90)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:98)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:97)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:299)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:258)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:54)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:81)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:81)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:50)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:369)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:457)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:477)
	at com.da

#### Configure access to the `psbs` blob storage account and `psdlsg2` data lake storage from Azure Databricks.

In [0]:
# Azure Synapse Analytics data warehouse related settings
dwHostname = "datatech.sql.azuresynapse.net" # this is the server name
dwPort = 1433 # default sql port
dwUsername = dbutils.secrets.get(scope = "datatech", key = "azure-sql-pwd") 
dwPassword = dbutils.secrets.get(scope = "datatech", key = "azure-sql-username") 
dwDatabase = "edw"

dwJdbcExtraOptions = "encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;"
sqlDwUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4};dwJdbcExtraOptions={5}".format(dwHostname,dwPort,dwDatabase,dwUsername,dwPassword,dwJdbcExtraOptions)

#### Read `JSON` from the  Data Lake Gen 2 Storage

In [0]:
mount_point = '/mnt/datatechstorage/users'
branch_data = sqlContext.read.json(f"{mount_point}/data/branch/*.json")
relations_managers_data = sqlContext.read.json(f"{mount_point}/data/relations_managers/*.json")

#### Prepare `Branch` Data

In [0]:
# create a temp table for the branch data to allow sql access
branch_data.createOrReplaceTempView('branch_data')

# display and explore the data
display(branch_data)
# observe that the branch name for the branch_id `1652` is empty. We will have to clean that up
# In addition, we have to type cast the date_created to a proper date type and rename columns for reporting

branch_id,branch_location,branch_name,date_created
1654,SYXGBYTPZG,,2011-02-20
1114,DTSDUCVRPFEGBD,ocnbnmajbwcmxx,2010-05-08
1025,FYPPUD,bqwvjydyjinvjm,2005-04-08
1759,XSXOQTRSDOJA,eyfbn,2014-05-04
1281,CQTJPLVYMCE,fiqexliefrj,2019-11-07
1250,QFZHFLBLCVQVLZT,emafcszcdbn,2018-02-03
1228,PC,ejhvlzqvqeyqjj,2020-11-29
1142,UY,ptpygczk,2015-05-20
1754,HGIENHHJESKBKD,tycyikr,2006-04-26
1104,ICTCMQINGC,cuvvevzswzptorq,2014-12-13


In [0]:
%sql
SELECT branch_id as BranchID
	,branch_location as BranchLocation
	,CASE 
		WHEN length(branch_name) = 0
			THEN lower(branch_location)||'-'||branch_id
		ELSE branch_name
		END AS BranchName
	,cast(date_created as date) as DateCreated
FROM branch_data

-- here we use sql to clean our data and display the final format as desired.
-- Now, obserce the empty branch name is replaced with the branch_location and the branch_id.
-- Also, all column names have been name according to the business reporting requirements

BranchID,BranchLocation,BranchName,DateCreated
1654,SYXGBYTPZG,syxgbytpzg-1654,2011-02-20
1114,DTSDUCVRPFEGBD,ocnbnmajbwcmxx,2010-05-08
1025,FYPPUD,bqwvjydyjinvjm,2005-04-08
1759,XSXOQTRSDOJA,eyfbn,2014-05-04
1281,CQTJPLVYMCE,fiqexliefrj,2019-11-07
1250,QFZHFLBLCVQVLZT,emafcszcdbn,2018-02-03
1228,PC,ejhvlzqvqeyqjj,2020-11-29
1142,UY,ptpygczk,2015-05-20
1754,HGIENHHJESKBKD,tycyikr,2006-04-26
1104,ICTCMQINGC,cuvvevzswzptorq,2014-12-13


In [0]:
branch = sql("""
SELECT branch_id as BranchID
	,branch_location as BranchLocation
	,CASE 
		WHEN length(branch_name) = 0
			THEN lower(branch_location)||'-'||branch_id
		ELSE branch_name
		END AS BranchName
	,cast(date_created as date) as DateCreated
FROM branch_data
""")

#### Write branch data to the data warehouse as a table

In [0]:
# Observe that the branch table is written as a REPLICATE type. 
# This is due to the small size and the need to be used as a dimension table

branch.write \
    .format("com.databricks.spark.sqldw") \
    .option("url", sqlDwUrl) \
    .option("useAzureMSI","true") \
    .option("forwardSparkAzureStorageCredentials", "true") \
    .option("dbtable", "branch") \
    .option("tableOptions", "CLUSTERED COLUMNSTORE INDEX, DISTRIBUTION = REPLICATE") \
    .mode("overwrite") \
    .save()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2878635156727351> in <module> 
 2 # This is due to the small size and the need to be used as a dimension table 
 3 
 ----> 4 branch . write \ 
 5 . format ( "com.databricks.spark.sqldw" ) \ 
 6 . option ( "url" , sqlDwUrl ) \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1132 self . format ( format ) 
 1133 if path is None : 
 -> 1134 self . _jwrite . save ( ) 
 1135 else : 
 1136 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o506.save.
: com.databricks.spark.sqldw.SqlDWConnectorException: Exception encountered in Azure Synapse Analytics connector code.
	at com.databricks.spark.sqldw.Utils$.wrapExceptions(Utils.scala:688)
	at com.databricks.spark.sqldw.DefaultSource.createRelation(DefaultSource.scala:89)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:96)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:196)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:240)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:236)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:192)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:165)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:164)
	at org.apache.spark.sql.DataFrameWriter.$anonfun$runCommand$1(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:126)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:267)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:852)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:217)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:468)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:438)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:311)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at s

#### Prepare `Relations Manager` Data

In [0]:
relations_managers_data.createOrReplaceTempView('relations_managers_data')
display(relations_managers_data)
# observe that the first_names for the relationship_manager_ids `503478`, `579131` and `595647` are empty. We will have to clean that up
# In addition, we have to type cast the last_updated to a proper date type and rename columns for reporting

In [0]:
%sql
SELECT branch_id AS BranchID
	,CASE 
		WHEN length(first_name) = 0
			THEN relationship_manager_id
		ELSE first_name
		END AS RmFirstName
	,last_name AS RmLastName
	,cast(last_updated AS DATE) AS LastUpdateDate
	,relationship_manager_id AS RmID
FROM relations_managers_data

In [0]:
relations_manager = sql("""
SELECT branch_id AS BranchID
	,CASE 
		WHEN length(first_name) = 0
			THEN relationship_manager_id
		ELSE first_name
		END AS RmFirstName
	,last_name AS RmLastName
	,cast(last_updated AS DATE) AS LastUpdateDate
	,relationship_manager_id AS RmID
FROM relations_managers_data
""")

#### Write relations manager data to the data warehouse as a table

In [0]:
# Notice how we overwrite these dimensions whenever we reload the table. 
# These are slow changing dimensions. Hence, we can afford to do so.

relations_manager.write \
    .format("com.databricks.spark.sqldw") \
    .option("url", sqlDwUrl) \
    .option("forwardSparkAzureStorageCredentials", "true") \
    .option("dbtable", "relations_manager") \
    .option("tableOptions", "CLUSTERED COLUMNSTORE INDEX, DISTRIBUTION = REPLICATE") \
    .mode("overwrite") \
    .option("tempdir", tempDir) \
    .save()